In [14]:
import os
import shutil
import cv2
from tqdm import tqdm

In [32]:
def bboxCVT2YOLO(size, box):
    dw = 1. / size[0]
    dh = 1. / size[1]

    # (xmin + xmax / 2)
    x = (box[0] + box[2]) / 2.0
    # (ymin + ymax / 2)
    y = (box[1] + box[3]) / 2.0

    # (xmax - xmin) = w
    w = box[2] - box[0]
    # (ymax - ymin) = h
    h = box[3] - box[1]

    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return (round(x, 3), round(y, 3), round(w, 3), round(h, 3))

In [20]:
image_path = '/local_hdd/works/ICRyu_workspace/for_yolo/dataset/data2/image_2'
label_path = '/local_hdd/works/ICRyu_workspace/for_yolo/dataset/data2/label_2'

save_path = '/local_hdd/works/ICRyu_workspace/for_yolo/dataset/data2/label_2_cvt'

In [3]:
label_list = os.listdir(label_path)

In [4]:
len(label_list)

7481

In [10]:
image_extensions = 'png'

## class name 불러오기

In [15]:
classes_list = []
for e in tqdm(label_list):
    with open(os.path.join(label_path, e), 'r') as f:
        labels_tmp = f.readlines()
        labels_tmp = list(map(lambda x: x.split(), labels_tmp))
        for l in labels_tmp:
            class_name = l[0]
            # if class_name == 'DontCare':
            #     continue
            if class_name not in classes_list:
                classes_list.append(class_name)

100%|██████████| 7481/7481 [00:00<00:00, 48385.62it/s]


In [16]:
classes_list

['Car',
 'Van',
 'Pedestrian',
 'Truck',
 'DontCare',
 'Misc',
 'Cyclist',
 'Person_sitting',
 'Tram']

In [40]:
yolo_class_dict = {
                'Car':'vehicle',
                'Van':'vehicle',
                'Pedestrian':'person',
                'Truck':'vehicle',
                'Misc':'twowheeler',
                'Cyclist':'twowheeler',
                'Person_sitting':'person'
}

In [41]:
yolo_class = ['person', 'vehicle', 'twowheeler']

In [42]:
each_file = label_list[0]

In [ ]:
# save_path 디렉토리 삭제
shutil.rmtree(save_path)
# save 디렉토리 없으면 생성하기
os.makedirs(save_path, exist_ok=True)

In [44]:
for each_file in tqdm(label_list):
    with open(os.path.join(label_path, each_file), 'r')  as f:
        img = cv2.imread(os.path.join(image_path, os.path.splitext(each_file)[0]+f'.{image_extensions}'), 1)
        img_h, img_w, img_c = img.shape

        # content = f.readlines()
        labels_tmp = f.readlines()
        labels_tmp = list(map(lambda x: x.split(), labels_tmp))
        labels = []

        for each_label in labels_tmp:
            class_name = each_label[0]
            truncated = float(each_label[1]) # 화면 밖으로 잘린 정도. 0 ~ 1
            occluded = int(each_label[2])    # 다른 물체에 의해 가려진 정도. 0=fully visible, 1=partly occluded, 2=largely occluded, 3=unknown
            if class_name == 'DontCare':
                continue
            if truncated >= 0.9:
                continue
            if occluded >= 3:
                continue

            x_min, y_min, x_max, y_max = list(map(lambda x: int(float(x)),each_label[4:8]))
            x_c, y_c, w, h = bboxCVT2YOLO([img_w, img_h], [x_min, y_min, x_max, y_max])
            if class_name not in yolo_class_dict:
                continue
            class_index = yolo_class.index(yolo_class_dict[class_name])
            labels.append(' '.join(list(map(str, [class_index, x_c, y_c, w, h]))))
            # labels.append(' '.join(list(map(str, [class_name, x_c, y_c, w, h]))))

        with open(os.path.join(save_path, each_file), 'w') as f:
            f.write('\n'.join(labels))




100%|██████████| 7481/7481 [01:03<00:00, 117.07it/s]


In [28]:
labels_tmp

[['Car',
  '0.00',
  '0',
  '-1.61',
  '589.09',
  '173.69',
  '649.29',
  '222.70',
  '1.36',
  '1.69',
  '3.38',
  '0.17',
  '1.42',
  '22.06',
  '-1.60'],
 ['Car',
  '0.78',
  '3',
  '-1.78',
  '0.00',
  '185.32',
  '125.39',
  '374.00',
  '1.45',
  '1.61',
  '3.96',
  '-5.72',
  '1.57',
  '5.81',
  '-2.53'],
 ['Car',
  '0.00',
  '1',
  '-1.93',
  '42.54',
  '190.42',
  '309.53',
  '334.07',
  '1.36',
  '1.57',
  '3.73',
  '-5.43',
  '1.59',
  '8.76',
  '-2.47'],
 ['Car',
  '0.00',
  '0',
  '1.10',
  '277.41',
  '173.84',
  '415.74',
  '245.98',
  '1.48',
  '1.57',
  '3.48',
  '-6.16',
  '1.53',
  '16.58',
  '0.75'],
 ['Car',
  '0.00',
  '0',
  '-1.38',
  '423.22',
  '186.10',
  '547.54',
  '261.21',
  '1.25',
  '1.74',
  '3.66',
  '-2.36',
  '1.54',
  '14.27',
  '-1.55'],
 ['Car',
  '0.00',
  '2',
  '-1.48',
  '482.21',
  '178.10',
  '553.59',
  '237.84',
  '1.46',
  '1.59',
  '3.65',
  '-2.46',
  '1.61',
  '19.64',
  '-1.61'],
 ['Car',
  '0.00',
  '2',
  '0.92',
  '406.03',
  '178